### 11. 가변 길이 인수(순전파 편)
- 입력이 여러 개이거나 출력이 여러 개일 수 있음
#### 1) Function 클래스 수정

In [34]:
import numpy as np

In [35]:
# 현재의 Function 클래스

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):
        raise NotImplementedError()

In [36]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [37]:
class Variable:
    def __init__(self, data):
        if data is not None:  # 여기 if문 추가
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # data와 같은 타입(float 등)의 1
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  # 함수 가져오기 (pop : 리스트의 마지막 원소 가져오기, 리스트에서는 삭제됨)
            x, y = f.input, f.output  # 함수의 입력과 출력을 가져옴
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [38]:
# 수정된 Function 클래스

class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
            
        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

#### 2) Add 클래스 구현

In [55]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return(y, )  # 튜플형태로 리턴

In [40]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


### 12. 가변 길이 인수(개선 편)
![](img/12_1.png)

In [54]:
class Function:
    def __call__(self, *inputs):  # 가변 길이 인수    
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
            
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0] 
                    # 원소가 하나면 첫번째 원소를 반환
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [49]:
# 가변 길이 인수 예제

def f(*x):
    print(x)
    
f(1,2,3)
f(1,2,3,4)

(1, 2, 3)
(1, 2, 3, 4)


In [56]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return(y, )  # 튜플형태로 리턴

In [57]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


#### 2) 함수를 구현하기 쉽도록
![](img/12_2.png)

In [111]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)  # 리스트 언팩 (self.forward(x0, x1)의 효과)
        if not isinstance(ys, tuple): # 튜플로
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
            
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0] 
                    # 원소가 하나면 첫번째 원소를 반환
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [112]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

#### 3) add 함수 구현

In [113]:
def add(x0, x1):
    return Add()(x0, x1)

In [114]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


### 13. 가변 길이 인수(역전파 편)
#### 1) Add 클래스의 역전파
![](img/13_1.png)
- x0에 대한 편미분값, x1에 대한 편미분값이 계산되어야!

In [216]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy     # x0 + x1의 편미분값은 1, 1이므로

In [217]:
def add(x0, x1):
    return Add()(x0, x1)

#### 2) Variable 클래스 수정

In [117]:
# 현재의 Variable 클래스

class Variable:
    def __init__(self, data):
        if data is not None:  
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output  # 이 부분 변경
            x.grad = f.backward(y.grad) # 이 부분 변경
            
            if x.creator is not None:
                funcs.append(x.creator)

In [118]:
class Variable:
    def __init__(self, data):
        if data is not None:  
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] # 여기서부터 수정한 부분
            gxs = f.backward(*gys)  # 인수에 *를 붙여 호출하면 리스트 언택됨
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
                
            for x, gx in zip(f.inputs, gxs):  # 미분값을 변수에 저장해줌
                x.grad = gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

#### 3) Square 클래스 구현

In [119]:
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.inputs[0].data # 수정된 부분 (수정전 : x = self.input.data)
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

In [120]:
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)

In [121]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


### 14. 같은 변수 반복 사용
#### 1) 문제점
- add(x, x)의 경우, 제대로 미분 X

In [122]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y :', y.data)

y.backward()
print('x.grad :', x.grad)

y : 6.0
x.grad : 1.0


In [123]:
# 문제의 원인

# class Variable:
#     [...]
#     def backward(self):
#         [...]
#         while funcs:
#             for x, gx in zip(f.inputs, gxs):
#                 x.grad = gx  # 이 부분이 덮어써짐

In [124]:
class Variable:
    def __init__(self, data):
        if data is not None:  
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:  # 수정한 부분
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

In [125]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

2.0


#### 2) 미분값 재설정
- 같은 변수를 사용하여 다른 계산을 할 경우 계산이 꼬이게 됨 <br/>
=> Variable 클래스에 미분값을 초기화하는 cleargrad 메서드 추가<br/>
    ○ 최적화 문제에서 유용 (최적화 문제 : 함수의 최솟값과 최댓값 찾는 문제)

In [130]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
5.0


In [127]:
class Variable:
    def __init__(self, data):
        if data is not None:  
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
                    
    def cleargrad(self):  # 추가한 부분
            self.grad = None

In [128]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산
x.cleargrad()  # 미분값 초기화
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


### 15. 복잡한 계산 그래프(이론 편)
- 다양한 위상의 계산 그래프에 대응하기 (위상 : 그래프의 연결된 형태)
- 현재는 아래의 그래프도 제대로 미분할 수 없음
![](img/15_1.png)
- 여기서, 변수 a에 2개의 미분값이 모두 전파된 '뒤'에야 a에서 x로 미분값을 전파할 수 있음 
- But, 현재는 c -> a -> x -> b -> a -> x 이렇게 됨

=> 처리할 함수의 '우선순위'를 부여해야! <br/>
    How? 함수와 변수의 '세대'를 기록하기
![](img/15_2.png)

### 16. 복잡한 계산 그래프(구현 편)

#### 1) 세대 추가

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0   # 세대 수를 기록하는 변수
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대를 기록한다 (부모세대 + 1)

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
                    
    def cleargrad(self):  # 추가한 부분
            self.grad = None

In [3]:
class Function(object):
    def __call__(self, *inputs):  # * 는 가변 길이 인수
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys] 
        
        self.generation = max([x.generation for x in inputs]) # 추가된 부분
        
        for output in outputs:
            output.set_creator(self)
            
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0] 
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

#### 2) 세대 순으로 꺼내기

In [4]:
generations = [2, 0, 1, 4, 2]
funcs = []

In [5]:
for g in generations:
    f = Function()  # 더미 함수 클래스
    f.generation = g
    funcs.append(f)

In [8]:
funcs

In [6]:
[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [10]:
funcs.sort(key=lambda x: x.generation)  # 리스트 정렬
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [11]:
f = funcs.pop()  # 가장 큰 값 꺼내기
f.generation

4

- '우선순위 큐'를 이용하면 더 효율적 (여기선 구현하지 않음)

#### 3) Variable 클래스의 backward

In [12]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0   # 세대 수를 기록하는 변수
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대를 기록한다 (부모세대 + 1)

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
            
        funcs = []   # (추가된 부분) 순서를 정렬할 함수들의 리스트
        seen_set = set()  # (함수 여러번 불리는 일 방지) 함수의 unique값
        
        def add_func(f):  # 함수 리스트를 세대 순으로 정렬하는 역할
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)  # 인수에 * 붙여 호출하면 리스트 언택됨
            if not isinstance(gxs, tuple): 
                gxs = (gxs, )
                
            for x, gx in zip(f.inputs, gxs): # x에 grad, func 넣어주기
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx   # 이미 grad가 있는 경우에 더해주기
                
                if x.creator is not None:
                    add_func(x.creator)  # 수정 전 : funcs.append(x.creator)
                    
    def cleargrad(self):  
            self.grad = None

In [89]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

In [27]:
print(y.data)
print(x.grad)

32.0
64.0


### 17. 메모리 관리와 순환 참조
- 메모리 관리의 두 가지 방식  
    ○ __참조 카운트__ : 참조 수를 세는 방식   
    ○ __GC__(Garbage Collection) : 세대를 기준으로 쓸모없는 객체 회수

    
#### 1~2) 참조 카운트
: 다른 객체가 참조할 때마다 1씩 증가 -> 참조가 끊길 때마다 1만큼 감소하다가 0이 되면 파이썬 인터프리터가 회수 


- 참조 카운트가 증가하는 경우  
    ○ 대입 연산자 사용   
    ○ 함수에 인수로 전달  
    ○ 컨테이너 타입 객체(리스트, 튜플, 클래스 등)에 추가시

In [2]:
# 참조 카운트 이해 예제 1

class obj:
    pass

def f(x):
    print(x)
    
a = obj()  # 변수에 대앱 : 참조 카운트 1
f(a)    # 함수에 전달 : 참조 카운트 2
# 함수 완료 : 빠져나오면 참조 카운트 1
a = None  # 대입 해제 : 참조 카운트 0  => 메모리에서 삭제

In [3]:
# 참조 카운트 이해 예제 2

a = obj()
b = obj()
c = obj()

a.b = b
b.c = c

a = b = c = None  # 아래의 오른쪽 그림과 같아진 후, 
# a가 없어지므로 b, c도 연쇄적으로 삭제됨

<img src='img/17_1.png' width='500'>

#### 3) 순환참조

In [6]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
c.a = a

a = b = c = None   # 아래의 오른쪽 그림과 같아짐
# 필요없는 객체이나 삭제되지 않음 => GC

<img src="img/17_2.png" width='400'>

#### 4) GC(generational garbage collection)
- 복잡하지만 영리하게 처리 (구조가 복잡하므로 설명 생략)
- 메모리가 부족한 시점에 자동으로 호출됨
- 하지만, 메모리 해제를 GC에 미루다 보면 메모리 사용량이 커짐 -> 순환 참조 만들지 않는게 좋음
- DeZero에서 '변수'와 '함수'를 연결하는 방식에 순환 참조가 숨어 있음  
=> weakref 모듈로 해결가능
<img src='img/17_3.png' width='200'>

#### 4) weakref 모듈
- 약한 참조 : 다른 객체를 참조하되 참조 카운트를 증가시키지 않는 기등

In [7]:
import weakref
import numpy as np

In [8]:
a = np.array([1, 2, 3])
b = weakref.ref(a)
b   # ndarray를 가리키는 약한 참조 (그림 17-1에서 참조카운트 1 1 1이 되는 것)

<weakref at 0x000002157F97E9F8; to 'numpy.ndarray' at 0x000002157F97E7B0>

In [9]:
b()  # 데이터에 접근하려면 b()라 써야함

array([1, 2, 3])

In [10]:
a = None
b   # ndarray가 삭제됨 (쥬피터는 사용자가 모르는 참조를 추가하므로 삭제가 되진 않음)

<weakref at 0x000002157F97E9F8; to 'numpy.ndarray' at 0x000002157F97E7B0>

##### DeZero에 추가하기

In [136]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]  # 약한 참조로
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [142]:
# Variable 클래스의 메서드에서도 수정해주기

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(Data)))
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = []
        seen_set = set()
        
        def add_funcs(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_funcs(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 
                # output 약한 참조해서 데이터에 접근하려면 output() 해줘야
            gxs = f.backward(*gys)  # 인수에 * 붙이면 리스트 언팩됨
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_funcs(x.creator)
                    
    def cleargrad(self):
        self.grad = None

#### 5) 동작확인
- for문이 두 번째 반복될 때 x와 y가 덮어 써짐 
- 이전의 계산 그래프는 더이상 참조 X -> 메모리 삭제됨

In [19]:
for i in range(10):
    x = Variable(np.random.randn(10000))  # 큰 데이터
    y = square(square(square(x)))   # 복잡한 계산

### 18. 메모리 절약 모드
① 불필요한 미분 결과 즉시 삭제  
② 역전파가 필요 없는 경우 고려

#### 1) 필요 없는 미분값 삭제

In [143]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 1.0
2.0 1.0


- 중간 변수들에 대한 미분값 제거하기

In [144]:
# Variable 클래스의 메서드에서도 수정해주기

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(Data)))
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
        
    def backward(self, retain_grad=False):  # 중간값 그레디언트 유지할지
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = []
        seen_set = set()
        
        def add_funcs(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_funcs(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 
                # output 약한 참조해서 데이터에 접근하려면 output() 해줘야
            gxs = f.backward(*gys)  # 인수에 * 붙이면 리스트 언팩됨
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_funcs(x.creator)
                    
            if not retain_grad:  # f.inputs의 grad만 남기고 f.outputs은 제거
                for y in f.outputs:
                    y().grad = None  # y는 약한 참조이므로  
                    
    def cleargrad(self):
        self.grad = None

In [145]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


#### 3) Config 클래스를 활용한 모드 전환
- (순전파만 할 경우를 위해) '역전파 활성 모드'와 '역전파 비활성 모드'를 전환하는 구조 필요 => Config 클래스

In [146]:
class Config:
    enable_backprop = True

In [147]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop:  # if문 안으로 넣으줌
            self.generation = max([x.generation for x in inputs]) # 세대 만들필요 X
            for output in outputs:  # 연결 해줄 필요 X in 역전파 비활성 모드
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
            
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

#### 4) 모드 전환

In [150]:
# class Square(Function):
#     def forward(self, x):
#         return x ** 2
    
#     def backward(self, gy):
#         x = self.inputs[0].data # 수정된 부분 (수정전 : x = self.input.data)
#         gx = 2 * x * gy
#         return gx

# def square(x):
#     return Square()(x)

In [151]:
Config.enable_backprop = True  # or False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()

#### 5) with 문을 활용한 모드 전환

In [ ]:
# with문 : 후처리(여기선 close)를 자동으로 수행

# f = open('sample.txt', 'w')
# f.write('hello world!')
# f.close

with open('sample.txt', 'w') as f:
    f.write('hello world!')

In [ ]:
# 앞으로 구현할 코드

with using_config('enable_backprop', False): # 이 안에서만 '역전파 비활성 모드'
    x = Variable(np.array(2.0))              # with 블록을 벗어나면 '활성 모드'
    y = square(x)

In [152]:
import contextlib

In [159]:
# contextlib 사용 방법

@contextlib.contextmanager  # 문맥을 판단하는 함수 만듦

def config_test():
    print('start')  # 전처리
    try:
        yield
    finally:
        print('done')  # 후처리
        
with config_test():
    print('process..')

start
process..
done


In [166]:
# using_config 함수 구현

@contextlib.contextmanager

def using_config(name, value):
    old_value = getattr(Config, name)  # Config.name 값
    setattr(Config, name, value)  # Config.name 값  value값으로 넣어주기
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [162]:
# getattr() 함수 설명

class sample:
    def __init__(self, x):
        self.x = x

c = sample(1)
getattr(c, 'x')  # c.x와 동일

1

In [167]:
# using_config 함수 사용해보기

with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

In [168]:
# with문 간단히하기

def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

In [171]:
y.data

array(4.)

### 19. 변수 사용성 개선
#### 1) 변수 이름 지정

In [173]:
class Variable:
    def __init__(self, data, name=None):  # name추가
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
#      [...]  
    
# x = Variable(np.array(1.0), 'input_x')라 작성하면
# 변수 x의 이름은 input_x가 됨 (이름 안주면 그냥 None)

#### 2) ndarray 인스턴스 변수
- Variable은 데이터를 담는 '상자' 역할인 데, Variable이 데이터인 것처럼 보이게 하는 장치만들어주기

In [174]:
# Variable에 shape 함수 추가하기

class Variable:
#     [...]
    @property  # shape 메서드를 인스턴스 변수처럼 사용할 수 있게
            # x.shape() 대신 x.shape으로 
        
    def shape(shape):
        return self.data.shape

In [175]:
# 다른 메서드들도 추가하기

class Variable:
#     [...]
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype

In [206]:
# len 함수와 print 함수 (19장 모두 포함)

class Variable:
    
    __array_prioirity__ = 200  # 큰 값으로 지정
    
    def __init__(self, data, name=None):  # name 추가
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
        
    def backward(self, retain_grad=False):  # 중간값 그레디언트 유지할지
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = []
        seen_set = set()
        
        def add_funcs(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_funcs(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 
                # output 약한 참조해서 데이터에 접근하려면 output() 해줘야
            gxs = f.backward(*gys)  # 인수에 * 붙이면 리스트 언팩됨
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_funcs(x.creator)
                    
            if not retain_grad:  # f.inputs의 grad만 남기고 f.outputs은 제거
                for y in f.outputs:
                    y().grad = None  # y는 약한 참조이므로  
                    # 최종 미분값은 출력해야하므로 (중간 미분값만 retain X)
                    # input의 미분값들은 제거 X
                    
    def cleargrad(self):
        self.grad = None

    @property  # shape 메서드를 인스턴스 변수처럼 사용할 수 있게
            # x.shape() 대신 x.shape으로 
    def shape(self):
        return self.data.shape
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
      
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):   # print 하면 variable(...) 형태로 출력하기
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9) # 다차원 array일 때 줄맞춰주려고
        return 'variable(' + p + ')'

### 20. 연산자 오버로드(1)

#### 1) Mul 클래스 구현
- $y=x_0 \times x_1$일 때, $\partial y / \partial x_0 = x_1, \partial y\partial x_1 = x_0$

In [229]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

In [230]:
def mul(x0, x1):
    return Mul()(x0, x1)

def add(x0, x1):
    return Add()(x0, x1)

In [190]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)  # y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


#### 2) 연산자 오버로드
- +와 * 바로 쓰도록 => 연산자 오버로드

In [197]:
class Variable:
#     [...]    
    def __mul__(self, other):  # * 연산자 사용시 __mul__ 메서드가 호출됨
        return mul(self, other)
    
# 아래의 코드가 가능해짐

In [198]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
y = a * b
print(y)

variable(6.0)


In [ ]:
# class Add(Function):
#     def forward(self, x0, x1):
#         y = x0 + x1
#         return y
    
#     def backward(self, gy):
#         return gy, gy     # x0 + x1의 편미분값은 1, 1이므로

# def add(x0, x1):
#     return Add()(x0, x1)

In [214]:
# 더 간단히 (클래스 정의 후)

Variable.__mul__ = mul   # 클래스내에서 __mul__ 함수 정의한 것과 동일한 효과
Variable.__add__ = add

In [209]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)  # y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


### 21. 연산자 오버로드(2)
- Variable 인스턴스, ndarray 인스턴스, int, float과 연산가능하도록

#### 1) ndarray와 함께 사용하기
- a * np.array(2.0) 가능하도록

In [210]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [211]:
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]  # as_variable 함수 추가해주기
        
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop:  # if문 안으로 넣으줌
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
                # 역전파 안 할거면 세대 만들고 연결해줄 필요 X
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
            
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [218]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


#### 2) float, int와 함께 사용하기

In [ ]:
# class Mul(Function):
#     def forward(self, x0, x1):
#         y = x0 * x1
#         return y
    
#     def backward(self, gy):
#         x0, x1 = self.inputs[0].data, self.inputs[1].data
#         return gy * x1, gy * x0

In [231]:
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

In [232]:
# 좌변이 float이나 int일 수 있으므로

Variable.__add__ = add
Variable.__mul__ = mul
Variable.__radd__ = add  # (float) + Variable 이면 연산자의 오른쪽에 위치한
Variable.__rmul__ = mul  # Variable에서 radd 메서드가 호출됨

In [233]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

variable(5.0)


In [234]:
x = Variable(np.array(2.0))
y = 3.0 * x + 1.0
print(y)

variable(7.0)


In [ ]:
# 좌항이 ndarray 인스턴스인 경우, 좌항에서 __add__ 메서드가 호출
# =>'연산자 우선순위'를 지정해야!

class Variable:
    __array_prioirity__ = 200  # 큰 값으로 지정
    [...]

### 22. 연산자 오버로드(3)
- 다른 연산자까지 확장
<img src='img/22_1.png' width='250'>

- 새로운 연산자 추가하는 순서  
    ① Function 클래스를 상속하여 원하는 함수 클래스 구현 (ex. Mul 클래스)  
    ② 파이썬 함수로 사용할 수 있도록 (ex. mul 함수)  
    ③ Variable 클래스의 연산자를 오버로드 (ex. Variable.__mul__ = mul)

#### 1) 부호 변환
- $y=-x$일 때, $\partial y / \partial x = -1$

In [236]:
class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy
    
def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

In [237]:
x = Variable(np.array(2.0))
y = -x
print(y)

variable(-2.0)


#### 2) 뺄셈
- $y=x_0-x_1$일 때, $\partial y / \partial x_0 = 1, \partial y / \partial x_1 = -1$

In [238]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y
    
    def backward(self, gy):
        return gy, -gy
    
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

Variable.__sub__ = sub

In [240]:
# __rsub__는 제대로 처리 못함 (순서를 바꿔서 계산하므로)
# => rsub 함수 만들기

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)  # 순서 바꾸기

Variable.__rsub__ = rsub

In [241]:
x = Variable(np.array(2.0))
y1 = 2.0 - x
y2 = x - 1.0

print(y1)
print(y2)

variable(0.0)
variable(1.0)


#### 3) 나눗셈
- $y=x_0/x_1$일 때, $\partial y=\partial x_0 = 1/x_1, \partial y=\partial x_1=-(x_0)^2/x_1^2$

In [242]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1
    
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)

Variable.__truediv__ = div
Variable.__rturediv__ = rdiv

#### 4) 거듭제곱
- $y=x^c$일 때, $\partial y/\partial x=cx^{c-1}$

In [243]:
class Pow(Function):
    def __init__(self, c):
        self.c = c
        
    def forward(self, x):
        y = x ** self.c
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c-1) * gy
        return gx
    
def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow

In [244]:
x = Variable(np.array(2.0))
y = x ** 3
print(y)

variable(8.0)


### 23. 패키지로 정리

In [7]:
# step22 파일을 dezero폴더의 core_simple.py로 옮기고
# __init__ 파일도 dezero 폴더에 넣어줌

if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

In [8]:
import numpy as  np
from dezero import Variable

In [9]:
x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

In [10]:
print(y)
print(x.grad)

variable(16.0)
variable(8.0)


### 24. 복잡한 함수의 미분

#### 1) Sphere 함수 
$z=x^2+y^2$

In [12]:
def sphere(x, y):
    z = x ** 2 + y ** 2
    return z

In [13]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = sphere(x, y)

z.backward()
print(x.grad, y.grad)

variable(2.0) variable(2.0)


#### 2) matyas 함수 
$z=0.26(x^2+y^2)-0.48xy$

In [14]:
def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z

In [15]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = matyas(x, y)
z.backward()

In [16]:
print(x.grad, y.grad)

variable(0.040000000000000036) variable(0.040000000000000036)


#### 3) Goldstein-Price 함수
<img src='img/24_1.png' width='300'>

In [17]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

In [18]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()

In [19]:
print(x.grad, y.grad)

variable(-5376.0) variable(8064.0)


### 4) 정적 계산 그래프(Define-and-Run) vs 동적 계산 그래프(Define-by-Run)

##### Define-and-Run
- 계산 그래프를 정의한 다음, 데이터를 흘려보낸다
<img src='img/24_2.png' width='300'>

- 계산 그래프 정의부분에서는 실제 계산이 이루어지지 않음. 기호를 대상으로 프로그래밍 됨 (기호 프로그래밍)

##### Define-by-Run
- 데이터를 흘려보냄으로써 계산 그래프가 정의됨

<img src='img/24_3.png' width='400'>


- 성능이 중요할 때는 Define-and-Run이 유리하고, 사용성이 중요할 때는 Define-by-Run이 훨씬 유리함 (텐서플로와 파이토치 등은 두 모드 모두를 지원함)